In [2]:
from google.colab import drive


drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
!pip install "datasets<3.0.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.6.1 which is incompatible.


In [4]:
# 1. Force uninstall both to clear the conflict
!pip uninstall -y unsloth unsloth_zoo

# 2. Install Unsloth Zoo DIRECTLY from Git (to get 'tiled_mlp')
!pip install --upgrade --no-cache-dir "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git"

# 3. Install Unsloth main library from Git
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-install-gfywxbd9/unsloth-zoo_ef49863a849f4fe19d5c9c75c962f350
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth-zoo.git /tmp/pip-install-gfywxbd9/unsloth-zoo_ef49863a849f4fe19d5c9c75c962f350
  Resolved https://github.com/unslothai/unsloth-zoo.git to commit bb2375ec6ebe00c65cfffc8f533a14b293646d11
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 245.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 353.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 404.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 390.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB

In [5]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.6 MB/s eta 0:00:00


In [6]:
from datasets import load_dataset


ds = load_dataset("xsum", trust_remote_code=True)
ds= ds.filter(lambda example: len(example["document"]) < 1500)
split_result = ds["test"].train_test_split(test_size=2, seed=42)
shots = split_result["test"]
ds["test"] = split_result["train"]

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'xsum' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'xsum' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of yo

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/300M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11332 [00:00<?, ? examples/s]

Filter:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [7]:
ft_ds = ds


def merge_columns(example):
    example["text"] = (
        f"[USER] Summarize this text. Don't hallucinate. Be thorough and explicit. Always provide an anwer. [TEXT]{example['document']}[AGENT]{example['summary']}"
    )
    return example

ft_ds = ft_ds.map(merge_columns)
from unsloth import FastLanguageModel

ft_model, ft_tokenzier = FastLanguageModel.from_pretrained(
    model_name="Qwen/Qwen1.5-0.5B", max_seq_length=4096, load_in_4bit=False
)

Map:   0%|          | 0/32053 [00:00<?, ? examples/s]

Map:   0%|          | 0/1746 [00:00<?, ? examples/s]

Map:   0%|          | 0/1727 [00:00<?, ? examples/s]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Qwen/Qwen1.5-0.5B does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.


In [10]:
from trl import SFTTrainer, SFTConfig
ft_model = FastLanguageModel.get_peft_model(
    ft_model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=64,
    lora_dropout=0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state=1234,
    use_rslora = False,
    loftq_config = None,

)
trainer = SFTTrainer(
    model=ft_model,
    train_dataset=ft_ds["train"],
    eval_dataset=ft_ds["validation"],
    tokenizer=ft_tokenzier,
    args=SFTConfig(
        max_seq_length=4096,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_steps=10,
        num_train_epochs=2,
        logging_steps=200,
        output_dir="outputs",
        optim="adamw_8bit",
        seed=3407,
        eval_strategy="steps",
        eval_steps=200,
    ),
)

Unsloth: Already have LoRA adapters! We shall skip this step.


🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [11]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 32,053 | Num Epochs = 2 | Total steps = 2,004
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 7,569,408 of 471,557,120 (1.61% trained)


Step,Training Loss,Validation Loss
200,2.247800,2.264317
400,2.245900,2.240530
600,2.226700,2.225239
800,2.215300,2.214223
1000,2.207900,2.204775
1200,2.149300,2.200292
1400,2.156100,2.195950
1600,2.141900,2.192628
1800,2.144900,2.190098
2000,2.147500,2.188936


TrainOutput(global_step=2004, training_loss=2.188244930046523, metrics={'train_runtime': 9568.7802, 'train_samples_per_second': 6.699, 'train_steps_per_second': 0.209, 'total_flos': 1.8004799597666304e+16, 'train_loss': 2.188244930046523, 'epoch': 2.0})

In [12]:
ft_model.save_pretrained('./drive/MyDrive/models/ft')